In [1]:
import cv2
import numpy as np
import json
import os
from matplotlib import pyplot as plt


In [ ]:
# Load MPII annotations
with open("path_to_mpii_annotations.json", "r") as f:
    annotations = json.load(f)

# Define a function to preprocess and load images and keypoints
def load_data(annotations, image_folder="path_to_images"):
    images = []
    keypoints_list = []
    for ann in annotations:
        # Load image
        img_path = os.path.join(image_folder, ann["image_id"] + ".jpg")
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Resize image (optional, to maintain consistency)
        img = cv2.resize(img, (256, 256))
        
        # Normalize keypoints to resized image
        keypoints = np.array(ann["keypoints"]).reshape(-1, 3)  # shape (16, 3)
        
        images.append(img)
        keypoints_list.append(keypoints)
    
    return images, keypoints_list

images, keypoints_list = load_data(annotations)


In [ ]:
def augment_data(img, keypoints):
    # Flip image horizontally
    flipped_img = cv2.flip(img, 1)
    flipped_keypoints = keypoints.copy()
    flipped_keypoints[:, 0] = img.shape[1] - keypoints[:, 0]  # Mirror x-coordinates
    
    # Return both original and augmented data
    return [(img, keypoints), (flipped_img, flipped_keypoints)]


In [ ]:
# Helper function to calculate angle between three points
def calculate_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Pose Detection Logic
def detect_pose(keypoints):
    # Define keypoints for specific joints
    left_hip, left_knee, left_ankle = keypoints[11], keypoints[12], keypoints[13]
    right_hip, right_knee, right_ankle = keypoints[8], keypoints[9], keypoints[10]
    
    # Walking Detection Logic
    if abs(left_knee[1] - right_knee[1]) > 20 and abs(left_ankle[0] - right_ankle[0]) > 10:
        return "Walking"
    
    # Squatting Detection Logic
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    if left_knee_angle < 90 or right_knee_angle < 90:
        return "Squatting"
    
    # Jumping Detection Logic
    if left_knee[1] < 100 and right_knee[1] < 100:  # Replace 100 with appropriate threshold
        return "Jumping"
    
    return "Unknown Pose"


In [ ]:
def visualize_keypoints(img, keypoints):
    for x, y, visibility in keypoints:
        if visibility > 0:  # Visible keypoint
            cv2.circle(img, (int(x), int(y)), 5, (0, 255, 0), -1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()


In [ ]:
for img, keypoints in zip(images, keypoints_list):
    pose = detect_pose(keypoints)
    
    # Visualize image with keypoints
    visualize_keypoints(img.copy(), keypoints)
    
    # Display detected pose
    print(f"Detected Pose: {pose}")


In [ ]:
cap = cv2.VideoCapture(0)  # 0 for default camera

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame to detect pose (you would need real-time keypoint detection for this)
    # Here, we would use a pre-trained model to predict keypoints in real-time
    keypoints = predict_keypoints(frame)  # Replace with actual keypoint prediction
    
    pose = detect_pose(keypoints)
    
    # Visualize and display pose
    visualize_keypoints(frame, keypoints)
    cv2.putText(frame, f"Pose: {pose}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    cv2.imshow("Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
